<a href="https://colab.research.google.com/github/abhiram212/Resume/blob/main/Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install necessary libraries
!pip install neo4j pandas numpy matplotlib plotly py2neo

# Step 2: Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from neo4j import GraphDatabase
from py2neo import Graph

# Step 2: Set Neo4j connection
uri = "neo4j+s://40eee460.databases.neo4j.io"
username = "neo4j"
password = "hvMg8ZEdGfpcoVII-o5uL4JrHyyvSskncn7FChEYjmw"

# Step 3: Load CSV data into a DataFrame
data = pd.read_csv('financial_transactions.csv')
print(data.head())
df = pd.DataFrame(data)

# Initialize Neo4j driver
driver = GraphDatabase.driver(uri, auth=(username, password))

# Step 4: Insert transaction data into Neo4j
def create_transaction(tx, transaction_id, timestamp, amount, transaction_type, account_id):
    query = """
    CREATE (t:Transaction {transaction_id: $transaction_id, timestamp: $timestamp,
                           amount: $amount, transaction_type: $transaction_type,
                           account_id: $account_id})
    """
    tx.run(query, transaction_id=transaction_id, timestamp=timestamp, amount=amount,
           transaction_type=transaction_type, account_id=account_id)

# Insert transactions into Neo4j
with driver.session() as session:
    for _, row in df.iterrows():
        session.write_transaction(create_transaction, row['transaction_id'], row['timestamp'],
                                  row['amount'], row['transaction_type'], row['account_id'])

# Step 5: Perform FMEA Analysis (Identifying Failure Modes)
# FMEA uses 3 criteria: Severity (S), Likelihood (L), and Detection (D)
df['severity'] = np.where(df['amount'] > 2500, 10, 5)  # High amount transactions considered severe
df['likelihood'] = np.where(df['transaction_type'] == 'Transfer', 8, 4)  # Transfer has higher likelihood of failure
df['detection'] = np.where(df['amount'] > 2000, 7, 3)  # Higher amounts are harder to detect

# Step 5.1: Calculate Risk Priority Number (RPN)
df['RPN'] = df['severity'] * df['likelihood'] * df['detection']

# Step 5.2: Identify transactions with high RPN (potential failures)
high_rpn_transactions = df[df['RPN'] > 100]
print("High RPN (Potential Failures):")
print(high_rpn_transactions)

# Step 6: Visualize the Financial Transaction Graph with FMEA
# Step 6.1: Create a graph in Neo4j showing transactions and failure relationships
def create_failure_relationship(tx, transaction_id):
    query = """
    MATCH (t:Transaction {transaction_id: $transaction_id})
    WHERE t.RPN > 100
    CREATE (f:Failure {transaction_id: $transaction_id})
    MERGE (t)-[:HAS_FAILURE]->(f)
    """
    tx.run(query, transaction_id=transaction_id)

# Step 6.2: Create failure nodes and relationships in Neo4j
with driver.session() as session:
    for _, row in high_rpn_transactions.iterrows():
        session.write_transaction(create_failure_relationship, row['transaction_id'])

# Step 7: Visualize transactions and failure modes using Plotly
fig = px.scatter(df, x='timestamp', y='amount', color='RPN', title="Financial Transactions with FMEA")
fig.update_traces(marker=dict(size=12))
fig.show()

# Step 8: Query Neo4j for the graph of transactions with failure modes
def query_trades_with_failures(tx):
    query = """
    MATCH (t:Transaction)-[:HAS_FAILURE]->(f:Failure)
    RETURN t.transaction_id, t.timestamp, t.amount, t.transaction_type, t.account_id, f.transaction_id
    ORDER BY t.timestamp
    """
    result = tx.run(query)
    for record in result:
        print(record['t.transaction_id'], record['t.timestamp'], record['t.amount'], record['t.transaction_type'], record['t.account_id'])

# Query the database for failed transactions
with driver.session() as session:
    session.read_transaction(query_trades_with_failures)


   transaction_id            timestamp  amount transaction_type account_id
0               1  2025-03-20 00:00:00    1500         Transfer       ACC1
1               2  2025-03-20 01:00:00    2000          Payment       ACC1
2               3  2025-03-20 02:00:00    2500         Transfer       ACC2
3               4  2025-03-20 03:00:00    1800          Deposit       ACC1
4               5  2025-03-20 04:00:00    3000          Payment       ACC3


<ipython-input-8-a87b95210552>:38: DeprecationWarning:

write_transaction has been renamed to execute_write



High RPN (Potential Failures):
   transaction_id            timestamp  amount transaction_type account_id  \
0               1  2025-03-20 00:00:00    1500         Transfer       ACC1   
2               3  2025-03-20 02:00:00    2500         Transfer       ACC2   
4               5  2025-03-20 04:00:00    3000          Payment       ACC3   

   severity  likelihood  detection  RPN  
0         5           8          3  120  
2         5           8          7  280  
4        10           4          7  280  


<ipython-input-8-a87b95210552>:69: DeprecationWarning:

write_transaction has been renamed to execute_write



<ipython-input-8-a87b95210552>:89: DeprecationWarning:

read_transaction has been renamed to execute_read

